In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import urllib
import time
import os
import more_itertools
%matplotlib inline
# RTSP = os.environ.get('RTSP_ADDRESS')
# cap = cv2.VideoCapture(RTSP)
cap = cv2.VideoCapture(1)
# cap = cv2.VideoCapture("http://192.168.1.101:4747/video")

def empty(a):
    pass

cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters",640,240)
cv2.createTrackbar("Area","Parameters",200,250,empty)
cv2.createTrackbar("lower","Parameters",40,255,empty)
cv2.createTrackbar("upper","Parameters",220,255,empty)
cv2.createTrackbar("Blur_kernel","Parameters",5,5,empty)
cv2.createTrackbar("Dilation_kernel","Parameters",2,5,empty)

def split_values(values, threshold, min_items):
    result = []
    if len(values) != 0:
        prev = values[0]
    else:
        prev = 0
    last_cut = 0
    for i,x in enumerate(values[1:], start=1):
        if abs(x - prev) >= threshold:
            if i - last_cut >= min_items:
                result.append((last_cut, i-1))
            last_cut = i
        prev = x
    if len(values) - last_cut >= min_items:
        result.append((last_cut, len(values)-1))
    return result


def getContours(img,imgContour,ymin=200,ymax=375):
    contours,hierarchy = cv2.findContours(img,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)[-2:]
    rectangle_center = []
    for i in range(len(contours)):
        #Filter using area function
        area = cv2.contourArea(contours[i])
        areaMax = cv2.getTrackbarPos("Area","Parameters")
        # If third column value is NOT equal to -1 than its internal
        if hierarchy[0][i][3] != -1 and area < areaMax:
            # Draw the Contour
            cv2.drawContours(imgContour, contours, i, (255,0,255), 2)
            peri = cv2.arcLength(contours[i],True)
            approx = cv2.approxPolyDP(contours[i],0.02*peri,True)
            x,y,w,h = cv2.boundingRect(approx)
#             cv2.rectangle(imgContour,(x,y),(x+w,y+h),(0,255,0),1)
            rectangle_center.append(x+w/2)
    return sorted(rectangle_center)

def approximate(img,imgContour,rectangle_center,ymin=200,ymax=375):
    groups = split_values(rectangle_center,3,7)
    font = cv2.FONT_HERSHEY_SIMPLEX 
    comp = [np.round(np.mean(rectangle_center[groups[i][0]:groups[i][1]]),2) for i,j in enumerate(groups)]
    for i in comp:
        cv2.line(imgContour, (int(i), 0), (int(i), 640), (0, 0, 255), thickness=1)
    try:
        if len(comp) == 2 and (comp[1]-comp[0]) < 150:
            diff = (comp[1]-comp[0])
            x1 = int(comp[0]+0.6*diff)
            x2 = int(comp[0]+0.9*diff)
            cv2.rectangle(imgContour,(x1,ymin),(x2,ymax),(255,0,0),2)
            contours,hierarchy = cv2.findContours(img[ymin:ymax,x1:x2],cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)[-2:]
            cont = [i for i in range(len(contours)) if hierarchy[0][i][3] != -1 and cv2.contourArea(contours[i]) < cv2.getTrackbarPos("Area","Parameters")]
            cont_len = str(len(cont))
            cv2.putText(imgContour,text=cont_len,org=(10,100), fontFace=font,fontScale= 4,color=(255,255,255),thickness=2,lineType=cv2.LINE_AA)
            cv2.imshow('Detected holes',imgContour[ymin:ymax,int(x1):int(x2)])
    except:
        print('Invalid comp dimension!')

    
while(cap.isOpened()):
    
    ret, frame = cap.read()
    imgContour = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    size = cv2.getTrackbarPos("Blur_kernel","Parameters")
    img_blur = cv2.GaussianBlur(gray,(size,size),1)
        
    
    #Cany
    med_val = np.median(img_blur) 
    lower = cv2.getTrackbarPos("lower","Parameters")
    upper = cv2.getTrackbarPos("upper","Parameters")
    edges = cv2.Canny(image=img_blur, threshold1=lower , threshold2=upper)

    #Dilation process
    dsize = cv2.getTrackbarPos("Dilation_kernel","Parameters")
    imgDil = cv2.dilate(edges,np.ones((dsize,dsize)),iterations=1)
    
    #Color inverse
    ret,thresh1 = cv2.threshold(imgDil,100,255,cv2.THRESH_BINARY_INV)
    
    #get contours
    contours = getContours(thresh1,imgContour)
    
    #Approximate holes pattern
    app = approximate(thresh1,imgContour,contours)
    
    #Output images
    cv2.imshow('Reference photo',frame)
    cv2.imshow("Image bluring", img_blur)
    cv2.imshow('After Dilation process',imgDil)
    cv2.imshow('Contous detection',imgContour)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()